In [2]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv('resources/train.csv')

In [3]:
df.head()

,user_id,restaurant_id,type,timestamp
0,2,17782,check-in,17/10/2012 11:03
1,6219,22458,check-in,26/10/2012 14:34
2,3,54559,check-in,28/10/2012 13:06
3,4,3377,check-in,29/10/2012 20:21
4,4,27473,check-in,29/10/2012 20:56


In [4]:
df['score'] = df.apply(lambda row: 2 if row['type']=='check-in' else 1 , axis = 1)

In [5]:
train_data = df.drop(columns=['type','timestamp'])

In [6]:
def calculate_score(df):
    return df['score'].sum()

In [7]:
temp = train_data.groupby(['restaurant_id','user_id']).apply(calculate_score)

In [8]:
temp = pd.DataFrame(temp).reset_index()

In [9]:
temp.columns = ['restaurant_id', 'user_id', 'score']

In [10]:
temp.to_csv(path_or_buf='resources/filtered_train.csv')

In [27]:
n_users = temp['user_id'].nunique()
n_restaurants = temp['restaurant_id'].nunique()

In [45]:
score_matrix = np.zeros((n_users,n_restaurants))
for index,row in temp.iterrows():
    score_matrix[row[1]-1, row[0]-1] = row[2]

MemoryError: 

In [44]:
c = 0
for index,row in temp.iterrows():
    print(row)
    c+=1
    if (c > 10):
        break;

restaurant_id     2
user_id          14
score             1
Name: 0, dtype: int64
restaurant_id       2
user_id          2087
score               1
Name: 1, dtype: int64
restaurant_id       2
user_id          4625
score               1
Name: 2, dtype: int64
restaurant_id       2
user_id          4626
score               1
Name: 3, dtype: int64
restaurant_id       2
user_id          4826
score               1
Name: 4, dtype: int64
restaurant_id       2
user_id          6346
score               1
Name: 5, dtype: int64
restaurant_id       2
user_id          7487
score               1
Name: 6, dtype: int64
restaurant_id       2
user_id          8250
score               1
Name: 7, dtype: int64
restaurant_id       2
user_id          8352
score               1
Name: 8, dtype: int64
restaurant_id        2
user_id          10365
score                1
Name: 9, dtype: int64
restaurant_id        2
user_id          10568
score                1
Name: 10, dtype: int64
